Tested on [![colab](https://img.shields.io/badge/Google-Colab-blue?logo=Google%20Colab)](https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/Alpaca-CoT/04_test_resume_pr.ipynb)

In [1]:
!git clone -b resume https://github.com/utensil/Alpaca-CoT.git code

Cloning into 'code'...
remote: Enumerating objects: 787, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 787 (delta 70), reused 85 (delta 48), pack-reused 662
Receiving objects: 100% (787/787), 127.14 MiB | 21.82 MiB/s, done.
Resolving deltas: 100% (422/422), done.


In [2]:
!ln -s /content /workspace

In [3]:
!python -m pip install bitsandbytes
!python -m pip install datasets
!python -m pip install git+https://github.com/huggingface/transformers.git
!python -m pip install peft
!python -m pip install sentencepiece
!python -m pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 10.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 400.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.5 MB/s eta 0:

In [4]:
# download data git repository
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/QingyiSi/Alpaca-CoT data
# pull specify data and move it，change <alpaca/> to your preference
!cd /workspace/data && git config core.sparsecheckout true && git config lfs.fetchinclude alpaca/
!cd /workspace/data && git lfs pull
!cp /workspace/data/alpaca/*.json /workspace/code/data/
!ls -lh /workspace/code/data/

Cloning into 'data'...
remote: Enumerating objects: 2195, done.
remote: Counting objects: 100% (2172/2172), done.
remote: Compressing objects: 100% (2115/2115), done.
remote: Total 2195 (delta 156), reused 1617 (delta 0), pack-reused 23
Receiving objects: 100% (2195/2195), 21.86 MiB | 5.91 MiB/s, done.
Resolving deltas: 100% (162/162), done.
total 44M
-rw-r--r-- 1 root root  22M Apr 19 13:13 alpaca_data_cleaned.json
-rw-r--r-- 1 root root  22M Apr 19 13:13 alpaca_data.json
drwxr-xr-x 2 root root 4.0K Apr 19 13:11 formatted_cot_data
drwxr-xr-x 2 root root 4.0K Apr 19 13:11 origin_cot_data


In [7]:
!cd /workspace && git clone https://github.com/utensil/llm-playground.git
!python3 /workspace/llm-playground/helper/upload.py

Cloning into 'llm-playground'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 202 (delta 117), reused 129 (delta 55), pack-reused 0
Receiving objects: 100% (202/202), 2.31 MiB | 4.47 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Working directory changed to: /workspace/llm-playground/helper/..
Cloning https://huggingface.co/datasets/utensil/storage into local empty directory.
Nothing to upload, exiting...


In [8]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=a713831c9274c7e437c14f02c6bf158ab92d85ae7c7fead0f4e770114a567a10
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [12]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [20]:
!cd /workspace/llm-playground/storage/ && git lfs pull

In [21]:
!cp -r /workspace/llm-playground/storage/saved_models /workspace/code/

In [22]:
!rm -rf /workspace/code/saved_models/llama-7b-hf_alpaca/llama-7b-hf_alpaca

In [23]:
!ls -lhta /workspace/code/saved_models/llama-7b-hf_alpaca/checkpoint-780

total 49M
drwxr-xr-x 7 root root 4.0K Apr 19 13:35 ..
drwxr-xr-x 2 root root 4.0K Apr 19 13:34 .
-rw-r--r-- 1 root root 8.9K Apr 19 13:34 trainer_state.json
-rw-r--r-- 1 root root 3.6K Apr 19 13:34 training_args.bin
-rw-r--r-- 1 root root  15K Apr 19 13:34 rng_state.pth
-rw-r--r-- 1 root root  557 Apr 19 13:34 scaler.pt
-rw-r--r-- 1 root root  627 Apr 19 13:34 scheduler.pt
-rw-r--r-- 1 root root  17M Apr 19 13:34 pytorch_model.bin
-rw-r--r-- 1 root root  33M Apr 19 13:34 optimizer.pt


In [ ]:
# instruction finetuning: trigger model download, output cleaned
!cd /workspace/code && python uniform_finetune.py --model_type llama --model_name_or_path decapoda-research/llama-7b-hf --data alpaca --lora_target_modules q_proj v_proj --per_gpu_train_batch_size 4 --learning_rate 3e-4 --epochs 1 --report_to wandb

In [16]:
# instruction finetuning: from scratch
!cd /workspace/code && python uniform_finetune.py --model_type llama --model_name_or_path decapoda-research/llama-7b-hf --data alpaca --lora_target_modules q_proj v_proj --per_gpu_train_batch_size 4 --learning_rate 3e-4 --epochs 1 --report_to wandb


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories list

In [24]:
# instruction finetuning: from the latest checkpoint
!cd /workspace/code && python uniform_finetune.py --model_type llama --model_name_or_path decapoda-research/llama-7b-hf --data alpaca --lora_target_modules q_proj v_proj --per_gpu_train_batch_size 4 --learning_rate 3e-4 --epochs 3 --report_to wandb --resume_from_checkpoint


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories list

In [25]:
!ls -lhta /workspace/code/saved_models/llama-7b-hf_alpaca/checkpoint-351

total 49M
drwxr-xr-x 7 root root 4.0K Apr 19 13:35 ..
-rw-r--r-- 1 root root 4.2K Apr 19 13:34 trainer_state.json
-rw-r--r-- 1 root root 3.5K Apr 19 13:34 training_args.bin
-rw-r--r-- 1 root root  557 Apr 19 13:34 scaler.pt
-rw-r--r-- 1 root root  627 Apr 19 13:34 scheduler.pt
-rw-r--r-- 1 root root  15K Apr 19 13:34 rng_state.pth
-rw-r--r-- 1 root root  17M Apr 19 13:34 pytorch_model.bin
-rw-r--r-- 1 root root  33M Apr 19 13:34 optimizer.pt
drwxr-xr-x 2 root root 4.0K Apr 19 13:17 .


In [26]:
# instruction finetuning: from the specified checkpoint
!cd /workspace/code && python uniform_finetune.py --model_type llama --model_name_or_path decapoda-research/llama-7b-hf --data alpaca --lora_target_modules q_proj v_proj --per_gpu_train_batch_size 4 --learning_rate 3e-4 --epochs 2 --report_to wandb --resume_from_checkpoint /workspace/code/saved_models/llama-7b-hf_alpaca/checkpoint-351


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories list